# Flynn taxonomy

- SISD
SISD stands for Single Instruction Stream, Single Data Stream. A SISD system has a single uniprocessor, which executes a single instruction stream to operate on data stored in a single memory.
Example: uniprocessor, first superscalar processors

- SIMD
SIMD stands for Single Instruction Stream, Multiple Data Stream. A SIMD system has multiple processing units which all simultaneously do the same operation on multiple points in a data pool in a synchronized fashion. 
Example: GPU

- MISD
MISD stands for Multiple Instruction Stream, Single Data Stream. A MISD system has multiple processing units performing different operations on thesame data simultaneously. It can be used for reliability or fault tolerance, if thesame operation is being done multiple times on the same data
Example: fault tolerance, space shuttle flight control

- MIMD
MIMD stands for Multiple Instruction Stream, Multiple Data Stream. A MIMD system has many processing units which each perform operations independently from each other at the same time. That is, each unit can be doing differentoperations on different pieces of data at any given time.
Example: modern multicore CPU

## Relation betweet SIMD MIMD SPMD
for all of them, the processor has access to multiple data streams
they are not the same
SIMD only deals with a single instruction stream, while MIMD and SPMD both use multiple instruction streams

SPMD is a subcategory of MIMD

# Shared Memory vs Distributed Memory

## Shared Memory

### Pros
+ user friendly programming perspective to memory
+ data sharing between tasks is fast and uniform

### Cons
- no scalability between memory and CPUs
- if you want more power,

## Distributed Memory

### Pros
+ memory is scalable with the number of processors
+ this is cost effective, because lots of cheap, commodity hardware can be used as opposed to upgrading a monolithic, more expensive structure

### Cons
- data communication between processors is more difficult
- it is difficult to map existing data structures to the memory organization
- memory access times can be very different. For example, local access times vs over a network
  access times can be much slower compared to a MIMD system
- more overhead for the programmer. The programmer has to think about message passing


# Amdahl's law
If y fraction of a serial program cannot be parallelized, 1/y is an upper bound on the speedup of its parallel program

S(p) = T1/Tp <= 1/( x/p + 1 - x)

lim_p->inf <= 1/(1-x)

Tp = T1/p + T_overhead

Efficiency
E(p) = S(p)/p = T1 / (p * Tp)

linear/ideal speedup
S(p) = p

sublinear speedup (common)
S(p) < p

slowdown
S(p+1) < S(p)


let T_serial = N

T_overhead = sqrt(P)

find P such that there is 12% degradation compared to ideal speedup

-> so that means the efficiency is 0.88

0.88 = N/ (p * ((N/p) + sqrt(p)))
wolfram:
p =  (3/22)^(2/3) * N^(2/3)

 = 128 (3/11)^(2/3) 2^(1/3)
 = 67.82227413238058016390


# multithreaded program examples

In [ ]:
// min max
#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>

#define NUM_ELEMENTS 1000000

int thread_count;
int* data; // array of integer values

void* min_max_task(void* thread_id);
int min_value;
int max_value;

int* generate_values(int n){
    int* vec = malloc(n * sizeof int));
    for (int i=0; i<n; ++i){
        vec[i] = rand()  % NUM_ELEMENTS;
    }
    return vec;
}

int main(int argc, char* argv[]){
    pthread_t* thread_handles;
    thread_count = 1
    if (argc > 1){
        thread_count = strtol(argv[i], NULL, 10);
    }
    thread_handles = malloc(thread_count * sizeof(pthread_t));
    data = generate_values(NUM_ELEMENTS);
    // insert here
    pthread_mutex_init(&mutex, NULL);multithrea

    for (long thread=0; thread< thread_count; thread++){
        pthread_create(&thread_handles[thread], NULL, min_max_task, (void*) thread);
    }

    for(long thread=0; thread< thread_countl thread++){
        pthread_join(thread_handles[thread], NULL);
    }
    printf("Minumum value is %d\n", min_value);
    printf("Maximum value is %d\n", max_value)
}

// insert here
pthread_mutex_t mutex;

// remember to insert this in main function
// pthread_mutex_init(&mutex, NULL);

void* min_max_task(void* thread_id){
    int id = (int) thread_id;
    int start = id * NUM_ELEMENTS/thread_count;
    int end = (id+1) * NUM_ELEMENTS/thread_count -1;

    for (int i=start; i<=end; i++){
        pthread_mutex_lock(&mutex);
        if (data[i] < min_value){
            min_value = data[i];
        }
        if (max_value < data[i]){
            max_value = data[i];
        }
        pthread_mutex_unlock(&mutex);
    }
}

multiple executions of the program produce different results

can happen because of lack of synchronization mechanisms which don't ensure the integrity of the data being modified

need to use mutex locks/condition variables when  doing computation on arrays simultaneously


In [ ]:
pthread_mutex_t mutex;
pthread_cond_t condvar;

In [ ]:
// with rwlock
/*
Created by Di Niu on Jan 26, 2016
Demonstrate the use of Read-Write locks in a 
program to find the minimum value in a list.
*/
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <unistd.h>
#include <pthread.h>
#include "timer.h"

typedef struct {
	int readers;
	int writer;
	pthread_cond_t readers_proceed;
	pthread_cond_t writer_proceed;
	int pending_writers;
	pthread_mutex_t read_write_lock;
} mylib_rwlock_t;

const int MAX_INT  = 10000000;
int partial_list_size;
int min_value;

pthread_mutex_t mutex;
mylib_rwlock_t rwlock; 


void mylib_rwlock_init (mylib_rwlock_t *l) {
	l -> readers = l -> writer = l -> pending_writers = 0;
	pthread_mutex_init(&(l -> read_write_lock), NULL);
	pthread_cond_init(&(l -> readers_proceed), NULL);
	pthread_cond_init(&(l -> writer_proceed), NULL);
}


void mylib_rwlock_rlock(mylib_rwlock_t *l) {
	/* if there is a write lock or pending writers, perform condition wait, else increment count of readers and grant read lock */

	pthread_mutex_lock(&(l -> read_write_lock));
	while ((l -> pending_writers > 0) || (l -> writer > 0))
		pthread_cond_wait(&(l -> readers_proceed),
			&(l -> read_write_lock));
	l -> readers ++;
	pthread_mutex_unlock(&(l -> read_write_lock));
}

void mylib_rwlock_wlock(mylib_rwlock_t *l) {
	/* if there are readers or writers, increment pending writers count and wait. On being woken, decrement pending writers count and increment writer count */
	
	pthread_mutex_lock(&(l -> read_write_lock));
	while ((l -> writer > 0) || (l -> readers > 0)) {
		l -> pending_writers ++;
		pthread_cond_wait(&(l -> writer_proceed),
			&(l -> read_write_lock));
    	l -> pending_writers --;
	}
	l -> writer ++;
	pthread_mutex_unlock(&(l -> read_write_lock));
}

void mylib_rwlock_unlock(mylib_rwlock_t *l) {
	/* if there is a write lock then unlock, else if there are read locks, decrement count of read locks. If the count is 0 and there is a pending writer, let it through, else if there are pending readers, let them all go through */

	pthread_mutex_lock(&(l -> read_write_lock));
	if (l -> writer > 0)
		l -> writer = 0;
	else if (l -> readers > 0)
		l -> readers --;
	pthread_mutex_unlock(&(l -> read_write_lock));
	if ((l -> readers == 0) && (l -> pending_writers > 0))
		pthread_cond_signal(&(l -> writer_proceed));
	else if (l -> readers > 0)
		pthread_cond_broadcast(&(l -> readers_proceed));
}

void *find_min_rw(void *list_ptr) {
	int *partial_list_pointer, my_min, i;
	my_min = MAX_INT;
	partial_list_pointer = (int *) list_ptr;
	for (i = 0; i < partial_list_size; i++)
		if (partial_list_pointer[i] <= my_min)
			my_min = partial_list_pointer[i];
	// lock the mutex associated with min_value and update the variable as required 
	mylib_rwlock_rlock(&rwlock);
	
	if (my_min < min_value) {
		mylib_rwlock_unlock(&rwlock);
		mylib_rwlock_wlock(&rwlock);
		min_value = my_min;
	}
	
	// and unlock the mutex
	mylib_rwlock_unlock(&rwlock);
	pthread_exit(0);
}


int main(int argc, char* argv[]) {
   int       thread, i;  
   pthread_t* thread_handles;
   double start, finish, elapsed;
   long thread_count = strtol(argv[1], NULL, 10);
   printf("%ld threads\n", thread_count);

	int n = 1000000; // problem size
   	int *list = (int *)malloc(n*sizeof(int));//input array
	for (i = 0; i < n; i++)
		list[i] = 99;
	partial_list_size = n/thread_count;	//problem size for each thread
	min_value = MAX_INT;
	
   thread_handles = (pthread_t*) malloc (thread_count*sizeof(pthread_t)); 
   pthread_mutex_init(&mutex, NULL);
   mylib_rwlock_init(&rwlock);
   
   GET_TIME(start);
   for (thread = 0; thread < thread_count; thread++)  
      pthread_create(&thread_handles[thread], NULL,
          find_min_rw, (void*)&list[thread*partial_list_size]);  

   for (thread = 0; thread < thread_count; thread++) 
      pthread_join(thread_handles[thread], NULL); 
   GET_TIME(finish);
   elapsed = finish - start;
   printf("Time taken: %f seconds\n", elapsed);
   printf("Min value is: %d\n", min_value);

   pthread_mutex_destroy(&mutex);
   free(thread_handles);
   return 0;
}  /* main */

In [ ]:
// this is a barrier
#include <pthread.h>
typedef struct {
  pthread_mutex_t count_lock;
  pthread_cond_t ok_to_proceed;
  int count;
} mylib_barrier_t;

void mylib_init_barrier(mylib_barrier_t *b) {
  b->count = 0;
  pthread_mutex_init(&(b->count_lock), NULL);
  pthread_cond_init(&(b->ok_to_proceed), NULL);
}

void mylib_barrier(mylib_barrier_t *b, int num_threads) {
  pthread_mutex_lock(&(b->count_lock));
  b->count++;
  if (b->count == num_threads) {
    b->count = 0;
    pthread_cond_broadcast(&(b->ok_to_proceed));
  } else
    while (pthread_cond_wait(&(b->ok_to_proceed), &(b->count_lock)) != 0)
      ;
  pthread_mutex_unlock(&(b->count_lock));
}


In [ ]:
//producer/consumer with barrier
#include <pthread.h>
#include <stdio.h>

#define BSIZE 3
#define NUMITEMS 6

typedef struct {
  char buf[BSIZE];
  int occupied;
  int nextin, nextout;
  pthread_mutex_t mutex;
  pthread_cond_t more;
  pthread_cond_t less;
} buffer_t;
buffer_t buffer;

void *producer(void *);
void *consumer(void *);

#define NUM_THREADS 2
pthread_t tid[NUM_THREADS]; /* array of thread IDs */

int main(int argc, char *argv[]) {

  int i;

  pthread_cond_init(&(buffer.more), NULL);
  pthread_cond_init(&(buffer.less), NULL);
  pthread_mutex_init(&buffer.mutex, NULL);
  pthread_create(&tid[1], NULL, consumer, NULL);
  pthread_create(&tid[0], NULL, producer, NULL);
  for (i = 0; i < NUM_THREADS; i++)
    pthread_join(tid[i], NULL);

  printf("\nmain() reporting that all %d threads have terminated\n", i);
} /* main */

void *producer(void *parm) {

  char item[NUMITEMS] = "HELLO.";
  int i;

  printf("producer started.\n");

  for (i = 0; i < NUMITEMS; i++) {
    /* produce an item, one character from item[] */
    if (item[i] == '\0')
      break; /* Quit if at end of string. */

    pthread_mutex_lock(&(buffer.mutex));

    if (buffer.occupied >= BSIZE)
      printf("producer waiting.\n");
    while (buffer.occupied >= BSIZE)
      pthread_cond_wait(&(buffer.less), &(buffer.mutex));
    printf("producer executing.\n");

    buffer.buf[buffer.nextin] = item[i];
    buffer.nextin++;
    buffer.nextin %= BSIZE;
    buffer.occupied++;
    /* now: either buffer.occupied < BSIZE and buffer.nextin
       is the index of the next empty slot in the
       buffer, or buffer.occupied == BSIZE and
       buffer.nextin is the index of the next
       (occupied) slot that will be emptied by a consumer

       (such as buffer.nextin == buffer.nextout) */
    pthread_cond_signal(&(buffer.more));
    pthread_mutex_unlock(&(buffer.mutex));
  }
  printf("producer exiting.\n");
  pthread_exit(0);
}

void *consumer(void *parm) {
  char item;
  int i;
  printf("consumer started.\n");
  for (i = 0; i < NUMITEMS; i++) {
    /*Insert here*/

    pthread_mutex_lock(&buffer.mutex);
    if (buffer.occupied < 1)
      printf("consumer waiting.\n");
    while (buffer.occupied < 1)
      pthread_cond_wait(&buffer.more, &buffer.mutex);
    printf("consumer executing.\n");

    item = buffer.buf[buffer.nextout];
    printf("%c\n", item);
    buffer.nextout++;
    buffer.nextout %= BSIZE;
    buffer.occupied--;

    pthread_cond_signal(&buffer.less);
    pthread_mutex_unlock(&buffer.mutex);
  }
  printf("consumer exiting.\n");
  pthread_exit(0);
}
